In [ ]:
## Constraints
# 1. # of pianists, 2 hands, 5 fingers
# 2. time apart
# 3. spatial apart (in each hand cannot exceed one octave)
# 4. within one octave > same intensity > notes start at the same time > last for same period > not adjacent

In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from midiutil import MIDIFile
from pprint import pprint

from utils import *

import copy
import random

In [2]:
## Parameters
group_num = 21 * 2 #21 pianists, 2 hands each
max_note_per_group = 5 #5 fingers

#min_pitch_gap = not set
win_length = 50e-3
max_time_gap = 0.3
max_frame_gap = int(round(max_time_gap / win_length))

In [3]:
key_time = 50e-3*0.5
Fs = 48000

## picked keys
keys = scipy.io.loadmat('MATLAB_data/23Feb_jc_key_2octave.mat')['key']

##freq of notes on piano
notes_list = scipy.io.loadmat('notes_112.mat')['notes'] 

##freq of t-f components
freq_list = scipy.io.loadmat('freq_480_two_octave_down.mat')['f_cropped'] 

intensity_map = scipy.io.loadmat('MATLAB_data/jw_int.mat')['filtered_int_db']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 112

notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

notes = sorted(notes, key = lambda note: note.start)

new_notes = remove_repetitive(notes,note_num)
#pprint(new_notes)
separate_majors(new_notes)


In [4]:
pprint(new_notes)

[pitch = 70(time = 27-29), intensity sign = mp, intensity = 44.665, major = 2, channel = -1,
 pitch = 70(time = 27-27), intensity sign = p, intensity = 28.694, major = 2, channel = -1,
 pitch = 71(time = 27-27), intensity sign = p, intensity = 16.982, major = 1, channel = -1,
 pitch = 74(time = 27-27), intensity sign = mp, intensity = 36.411, major = 2, channel = -1,
 pitch = 74(time = 27-28), intensity sign = mp, intensity = 37.211, major = 2, channel = -1,
 pitch = 75(time = 27-27), intensity sign = p, intensity = 28.562, major = 1, channel = -1,
 pitch = 75(time = 27-32), intensity sign = mp, intensity = 49.785, major = 1, channel = -1,
 pitch = 75(time = 27-35), intensity sign = mf, intensity = 61.885, major = 1, channel = -1,
 pitch = 75(time = 27-29), intensity sign = mp, intensity = 45.031, major = 1, channel = -1,
 pitch = 75(time = 27-33), intensity sign = mp, intensity = 54.691, major = 1, channel = -1,
 pitch = 75(time = 27-31), intensity sign = mp, intensity = 40.446, major

In [5]:
word_gap = 15
def separate_word(notes,word_gap):
    word_list = []
    ptr = 0
    for i in range(len(notes)-1):
        if notes[i+1].start - notes[i].end >= word_gap:
            word_list.append(notes[ptr:i+1])
            ptr = i + 1
        if i == len(notes) - 2:
            word_list.append(notes[ptr:])
    return word_list
    

In [6]:
def valid_note_to_allocate(note,overlapped,pitch_group,high):
    low = note.pitch if pitch_group == [] else min(pitch_group)
    
    if (note.pitch not in pitch_group) and (note.pitch-low <= 12):
        #q remain: limit highest note range?
        if overlapped and note.pitch < (high + 1):
            return False
        return True
    return False

def valid_to_add(counter,candidate,current_group):
    c1 = counter < len(candidate)
    c2 = len(current_group) < 5
    return c1 and c2

In [7]:
def priority(note, current_allocated_note,major):
    p = 0
    # if note.major == major:
    #     p += 3

    if note.intensity_sign == current_allocated_note.intensity_sign:
        p += 4
    
    if note.start == current_allocated_note.start and note.end == current_allocated_note.end:
        p += 2
    
    return p

In [ ]:
def check_handshape(allocated_group,overlapped):
    n = len(allocated_group) #number of notes in the group
    if n < 3:
        return True
    if overlapped:
        #right hand
        if n == 3:
            return
        if n == 4:
            return
        if n == 5:
            return
    else:
        #left hand
        if n == 3:
            return
        if n == 4:
            return
        if n == 5:
            return
    print('Group not valid')
    return False

In [8]:
def allocation(notes_group):
    allocate_result=[]
    group = 1
    overlapped = False #true if last group was left hand, current group is right hand, they should not overlap
    while notes_group != [] and group <= group_num:
        ## each while iteration allocate one group
        current_group = []
        pitch_group = []
        
        if not overlapped:
            high = 0
            minnote = min(notes_group, key = lambda note: note.pitch)
            pitch_group = [minnote.pitch]
            major = minnote.major
            current_group.append(minnote)
            notes_group.remove(minnote)
        else:
            for i in range(len(notes_group)):
                if notes_group[i].pitch > (high + 1):
                    pitch_group = [notes_group[i].pitch]
                    current_group.append(notes_group.pop(i))
                    break
        candidate = []            
        for current_allocated_note in current_group:
            candidate = []
            # counter = 0
            for note in notes_group:
                if not valid_note_to_allocate(note,overlapped,pitch_group,high):
                    continue
                p = priority(note,current_allocated_note,major)
                candidate.append([p,note])
            candidate.sort(key = lambda x: x[0], reverse=True)
            #print(candidate)
            counter = 0
            while valid_to_add(counter,candidate,current_group):
                if  (candidate[counter][1].pitch not in pitch_group) and (candidate[counter][0] > 4):
                    pitch_group.append(candidate[counter][1].pitch)
                    current_group.append(candidate[counter][1])
                    notes_group.remove(candidate[counter][1])
                counter += 1

        counter = 0
        while valid_to_add(counter,candidate,current_group):
            if (candidate[counter][1].pitch not in pitch_group):
                pitch_group.append(candidate[counter][1].pitch)
                current_group.append(candidate[counter][1])
                if candidate[counter][1] not in notes_group:
                    print(candidate[counter][1])
                notes_group.remove(candidate[counter][1])
            counter += 1

        if not overlapped:
            high = max([n.pitch for n in current_group])

        allocate_result.append(current_group)

        overlapped = not overlapped
        group += 1 #finish allocate notes to one group, update
    
    counter = 0
    while counter < len(notes_group):
        note = notes_group[counter]
        for i, allocated_group in enumerate(allocate_result):
            if len(allocated_group) >= 4:
                continue
            if len(allocated_group) < 4:
                pitch_group = [n.pitch for n in allocated_group]
                high = max([n.pitch for n in allocate_result[i-1]]) if i % 2 == 0 else 0
                overlapped = True if i % 2 == 0 else False
                if valid_note_to_allocate(note,overlapped,pitch_group,high):
                    allocated_group.append(note)
                    notes_group.remove(note)
                    break
        counter += 1

    if notes_group != []:
        print('Failed :(    ',len(notes_group), 'notes unallocated.')
        return False
    else:
        print('Success!')
        return allocate_result

In [9]:
word_groups = separate_word(new_notes,word_gap)
for item in word_groups:
    print(len(item))

455
261
78
158
190


In [17]:
# word_groups = separate_word(new_notes)
final_allocation1 = [[],[]]
final_allocation2 = [[],[]]
result_all = []
for i in range(2,5):
    print(i)
    current_allocate_group = copy.deepcopy(word_groups[i])
    #random.shuffle(current_allocate_group)
    result = False
    while not result:
        result = allocation(current_allocate_group)
        current_allocate_group = copy.deepcopy(word_groups[i])
        random.shuffle(current_allocate_group)
    result_all.append(result)
    final_allocation1[0].extend(result[0])
    final_allocation1[1].extend(result[1])
    final_allocation2[0].extend(result[20])
    final_allocation2[1].extend(result[21])
    pprint(final_allocation1)
    pprint(final_allocation2)

    

2
Success!
[[pitch = 33(time = 118-118), intensity sign = mp, intensity = 36.148, major = 1, channel = -1,
  pitch = 35(time = 123-123), intensity sign = mp, intensity = 40.490, major = 1, channel = -1,
  pitch = 40(time = 126-132), intensity sign = mp, intensity = 52.687, major = 1, channel = -1,
  pitch = 42(time = 128-132), intensity sign = mp, intensity = 49.371, major = 1, channel = -1,
  pitch = 39(time = 118-118), intensity sign = p, intensity = 27.826, major = 1, channel = -1],
 [pitch = 47(time = 118-118), intensity sign = p, intensity = 29.089, major = 1, channel = -1,
  pitch = 49(time = 121-126), intensity sign = p, intensity = 31.546, major = 1, channel = -1,
  pitch = 56(time = 121-121), intensity sign = p, intensity = 27.526, major = 1, channel = -1,
  pitch = 57(time = 123-124), intensity sign = p, intensity = 25.931, major = 1, channel = -1,
  pitch = 51(time = 125-127), intensity sign = p, intensity = 30.014, major = 1, channel = -1]]
[[pitch = 52(time = 129-129), int

C:\Users\96326\AppData\Local\Temp\ipykernel_17776\239073358.py:4: RuntimeWarning: overflow encountered in ubyte_scalars
  if (note.pitch not in pitch_group) and (note.pitch-low <= 12):


Failed :(     12 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     5 notes unallocated.
Failed :(     7 notes unallocated.
Failed :(     1 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     7 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     5 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     7 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     2 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     8 notes unallocated.
Failed :(     7 notes unallocated.
Failed :(     10 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     5 notes unallocated.
Failed :(     8 notes unallocated.
Failed :(     6 notes unallocated.
Failed :(     7 notes unallocated.
Failed :(     4 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     3 notes unallocated.
Failed :(     9 no

In [ ]:
current_allocate_group = copy.deepcopy(word_groups[4])
result = False
while not result:
    result = allocation(current_allocate_group)
    current_allocate_group = copy.deepcopy(word_groups[4])
    random.shuffle(current_allocate_group)


In [ ]:
plot_notes(new_notes)
plt.show()
word_groups = separate_word(new_notes)
for g in word_groups:
    plot_notes(g)
plt.show()

In [14]:
mf = MIDIFile(2)     # track number
time = 0    # start at the beginning
mf.addTrackName(0, time, "Right")
mf.addTrackName(1, time, "Left")
mf.addTempo(0, time, 1200)
mf.addTempo(1, time, 1200)

In [15]:
for i in range(2):    
    for note in final_allocation2[1-i]:
        # v = int(50+note.sign()*12.5)
        v = int(20+note.sign()*20)
        mf.addNote(i,i,note.pitch+20, note.start, note.end-note.start+1,v)

In [16]:
with open("midifile/23Feb_allocation2.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
pprint(final_allocation)

In [ ]:
import numpy as np
array = np.array(result_all,dtype=object)
np.save("allocation_result.npy", array)